In [3]:
from pathlib import Path
import ross as rs
import numpy as np

###################################################
import plotly.io as pio
#pio.renderers.default = "notebook"
pio.renderers.default = "browser"
###################################################

# uncomment the lines below if you are having problems with plots not showing
# import plotly.io as pio
# pio.renderers.default = "notebook"

# The Main procedure of ROSS is "Building the model + Calculating the results".(모델링 + 결과 고찰하기가 핵심!)
# SI unit as default

In [ ]:
############################ 자재 저장 안되어 있을 경우만 실행 시키기 #######################

# Section 1. Material Class

# 1.1 Creating a material
# 새로운 material 을 입력하기 위해서 사용. E, G, rho 중에 2개 입력
# material안에 있는 "name"이 실제로 저장되는 이름

# from E and G_s
steel1 = rs.Material(name="Steel", rho=7810, E=211e9, G_s=81.2e9)

# material properties used for Chilbo No.2
brake_p = rs.Material(name="brake_mat", rho=7850, E=200e9, G_s=80e9)
lowfan_p = rs.Material(name="lowfan_mat",  rho=7850, E=200e9, G_s=80e9)
upfan_p = rs.Material(name="upfan_mat",  rho=7850, E=200e9, G_s=80e9)
thrcollar_p = rs.Material(name="thrcollar_mat",  rho=7850, E=200e9, G_s=80e9)
collarkey_p = rs.Material(name="collarkey_mat",  rho=7850, E=200e9, G_s=80e9)
slipring_p = rs.Material(name="slipring_mat", rho=7850, E=200e9, G_s=80e9)  # properties NOT provided
speedsensor_p = rs.Material(name="SSG_mat",  rho=7850, E=200e9, G_s=80e9)
rimkey_p = rs.Material(name="rimkey_mat",  rho=7850, E=200e9, G_s=80e9)


# returning attributes
print("="*36)
print(f"Young's Modulus: {brake_p.E}")
print(f"Shear Modulus:    {brake_p.G_s}")

# Saving materials
brake_p.save_material()
lowfan_p.save_material()
upfan_p.save_material()
thrcollar_p.save_material()
collarkey_p.save_material()
slipring_p.save_material()
speedsensor_p.save_material()
rimkey_p.save_material()

# 사용 가능한 material properties 저장 파일 위치 확인하기(2024.12.16)
print(rs.__file__)
print("="*36)
print(rs.Material.available_materials())
#####################################################################################################################

In [4]:
# 사용 가능한 material 불러오기
['Steel11', 'Steel', 'Steel_Laminated', 'brake_mat', 'lowfan_mat', 'upfan_mat', 'thrcollar_mat', 'collarkey_mat', 'slipring_mat', 'SSG_mat', 'rimkey_mat']


steel_pp = rs.Material.load_material('Steel')
brake_pp = rs.Material.load_material('brake_mat')
lowfan_pp = rs.Material.load_material('lowfan_mat')
upfan_pp = rs.Material.load_material('upfan_mat')
thrcollar_pp = rs.Material.load_material('thrcollar_mat')
collarkey_pp = rs.Material.load_material('collarkey_mat')
slipring_pp = rs.Material.load_material('slipring_mat')
SSG_pp = rs.Material.load_material('SSG_mat')
rimkey_pp = rs.Material.load_material('rimkey_mat')


In [13]:
#### Disk list 만들기

# 이렇게 station number를 직접 적을때는 station number가 0부터 시작했을 때 규칙을 기반으로 적기 #


# common properites
gr = 9.8 # unit: m2/s


# runner on station 1
runner = 21364      # unit: N
runner_l = 0.3947   # unit: m
runner_m = 2200    #unit: kg
runner_Ip = 24427/(4*gr)    #unit : kg*m2
runner_It = 0.5*runner_Ip + (runner_m*(runner_l)**2)/12    #unit : kg*m2
runner_Ith = runner_It + runner_m*(runner_l/2)**2

print(f"runner_It on c.g.:{runner_It}")



# 직접 inertia properties를 이용한 디스크 만들기
runner_disk = rs.DiskElement(
    n=1-1,            # node to input must be inserted.(or an error occurs!)
    m=runner_m,
    Ip=runner_Ip,
    Id=runner_Ith,
    tag="runner_disk"  # A tag to name the element Default is None
)
print(runner_disk)


runner_It on c.g.:340.1300273843537
Tag:                      runner_disk
Node:                     0
Mass           (kg):      2200.0
Diam. inertia  (kg*m**2): 425.81
Polar. inertia (kg*m**2): 623.14


In [16]:
# Ith를 다 맞춰야됨
# 질량도 다 맞춰야됨 그거 빼고함


brake_disk = rs.DiskElement.from_geometry(
    n=32,   # plot 보고 수정한 station #
    material=brake_pp,
    width=210*0.001,
    i_d=1070*0.001,
    o_d=1730*0.001,   
)
print(brake_disk) # Ith를 다 맞춰야됨

lowfan_disk = rs.DiskElement.from_geometry(
    n=32,
    material=lowfan_pp,
    width=51*0.001,
    i_d=1613*0.001,
    o_d=2100*0.001,    
)
print(lowfan_disk) # Ith를 다 맞춰야됨

upfan_disk = rs.DiskElement.from_geometry(
    n=40,
    material=upfan_pp,
    width=51*0.001,
    i_d=1613*0.001,
    o_d=2100*0.001,    
)
print(upfan_disk) # Ith를 다 맞춰야됨

thrcollar_disk = rs.DiskElement.from_geometry(
    n=49-1,
    material=thrcollar_pp,
    width=824*0.001,
    i_d=500*0.001,
    o_d=710*0.001,    
)
print(thrcollar_disk) # Ith를 다 맞춰야됨

collarkey_disk = rs.DiskElement.from_geometry(
    n=51-1,
    material=collarkey_pp,
    width=50*0.001,
    i_d=388*0.001,
    o_d=600*0.001,     # CAD로 다시 확인하기    
)
print(collarkey_disk)

slipring_disk1 = rs.DiskElement.from_geometry(
    n=56-1,
    material=slipring_pp,
    width=31*0.001,
    i_d=130*0.001,
    o_d=600*0.001,    
)
print(slipring_disk1)

slipring_disk2 = rs.DiskElement.from_geometry(
    n=57-1,
    material=slipring_pp,
    width=39*0.001,
    i_d=130*0.001,
    o_d=600*0.001,    
)
print(slipring_disk2)

slipring_disk3 = rs.DiskElement.from_geometry(
    n=58-1,
    material=slipring_pp,
    width=39*0.001,
    i_d=130*0.001,
    o_d=600*0.001,    
)
print(slipring_disk3)


## 시스템 오류라서 수정이 불가
SSG_disk = rs.DiskElement.from_geometry(
    n=59,
    material=SSG_pp,     # SSG_mat 입력 필요
    width=1730*0.001,
    i_d=130*0.001,
    o_d=300*0.001,    # CAD로 다시 확인하기
)
print(SSG_disk)

print("="*36)
list_of_disks = [runner_disk, brake_disk, lowfan_disk, upfan_disk, thrcollar_disk, collarkey_disk, slipring_disk1, slipring_disk2, slipring_disk3, SSG_disk]
print(list_of_disks)

Tag:                      None
Node:                     32
Mass           (kg):      2392.7
Diam. inertia  (kg*m**2): 627.56
Polar. inertia (kg*m**2): 1237.5
Tag:                      None
Node:                     32
Mass           (kg):      568.57
Diam. inertia  (kg*m**2): 249.29
Polar. inertia (kg*m**2): 498.33
Tag:                      None
Node:                     40
Mass           (kg):      568.57
Diam. inertia  (kg*m**2): 249.29
Polar. inertia (kg*m**2): 498.33
Tag:                      None
Node:                     48
Mass           (kg):      1290.9
Diam. inertia  (kg*m**2): 133.88
Polar. inertia (kg*m**2): 121.68
Tag:                      None
Node:                     50
Mass           (kg):      64.569
Diam. inertia  (kg*m**2): 2.0738
Polar. inertia (kg*m**2): 4.1206
Tag:                      None
Node:                     55
Mass           (kg):      65.576
Diam. inertia  (kg*m**2): 1.55
Polar. inertia (kg*m**2): 3.0894
Tag:                      None
Node:            

In [17]:
### shaft의 lamination하는 경우 어떻게 element 입력하는 지 확인함

## Tips on shaft element building via Excel
# 1. Element number는 반드시 1부터 시작해야 된다.(0으로 시작하면 인식 못함) 따라서, Disk 등 added mass 넣을 때 번호 주의
# 2. Layer를 입힐 수 있음. Layer는 대부분 stiffness가 0이므로, rho만 넣어 주면 된다.
# 3. 그러나 Layer중에 stiffness가 약간 있는 경우는(실험 등에서 구한 경우) 기존 modulus에서 엄청  %를  줄여서 input으로 넣는다.(위의 material #2 경우)


## load the shaft from an excel sheet
shaft_file = Path("shaft_si6.xls")

shaft = rs.ShaftElement.from_table(
    file=shaft_file, sheet_type="Model", sheet_name="Model"
)

#shaft
# print(shaft[0])       # 1개 shaft[0] element의 properties 표시
#shaft[0].M().shape

#file_path = Path("shaft_si6.xls")
#list_of_disks = rs.DiskElement.from_table(file=file_path, sheet_name="More")



## rotor generation with shaft and list of disks
rotor1 = rs.Rotor(shaft, list_of_disks)

#rotor1 = rs.Rotor(shaft)
rotor1.M(0).shape


## node 표시를 5개 마다 표시

node_increment = 5
rotor1.plot_rotor(nodes=node_increment)
# default는 node_increment = 1
# rotor1.plot_rotor()

In [ ]:
shaft_file = Path("shaft_si.xls")
shaft = rs.ShaftElement.from_table(
    file=shaft_file, sheet_type="Model", sheet_name="Model"
)

file_path = Path("shaft_si.xls")
list_of_disks = rs.DiskElement.from_table(file=file_path, sheet_name="More")

bearing1 = rs.BearingElement.from_table(n=7, file="bearing_seal_si.xls")
bearing2 = rs.BearingElement.from_table(n=48, file="bearing_seal_si.xls")

bearings = [bearing1, bearing2]

rotor3 = rs.Rotor(shaft, list_of_disks, bearings)

rotor3.plot_rotor()